In [2]:
import pandas as pd
import dspy
import os
from dspy.evaluate import Evaluate
from dspy.evaluate.metrics import answer_exact_match

In [26]:
# Load the CSV dataset
df = pd.read_csv("agent_decision_dataset.csv")

In [27]:
# Shuffle the dataframe
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert to list of dictionaries
dataset_dict = df_shuffled.to_dict(orient='records')

# Split dataset 
split_index = int(len(dataset_dict) * 0.8)
trainset = dataset_dict[:split_index]
testset = dataset_dict[split_index:]

train_examples = [
    dspy.Example(Query=x["Query"], Type=x["Type"]).with_inputs("Query")
    for x in trainset
]

test_examples = [
    dspy.Example(Query=x["Query"], Type=x["Type"]).with_inputs("Query")
    for x in testset
]


In [28]:
class QueryTypePrediction(dspy.Signature):
    """
    You are an intent classifier for Bookme.pk. 

    Possible outputs:
      - 'static' if the query is about FAQs, policies, or general information.
      - 'api' if the query requests live data.

    Always respond with ONLY 'static' or 'api'.
    """
    Query = dspy.InputField(desc="The user query")
    Type = dspy.OutputField(desc="The type of query: static or api")

# Module for Query -> Type prediction
class QueryTypePredictor(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate = dspy.ChainOfThought("Query -> Type")
    
    def forward(self, Query):
        return self.generate(Query=Query)


In [6]:
llm = dspy.LM("groq/llama3-8b-8192", api_key="")
dspy.settings.configure(lm=lm, max_tokens=1024, temperature=0)

In [30]:
class Assess(dspy.Signature):
    """
    Decide if predicted type (api/static) fits the query.
    """
    Query = dspy.InputField(desc="The user's query for Bookme.pk")
    Type = dspy.InputField(desc="Predicted type of query: 'api' or 'static'")
    verdict = dspy.OutputField(desc="Return 'YES' if Type fits the query intent. Otherwise return 'NO'.")

def metric(example, prediction, trace=None):
    assessment = dspy.Predict(Assess, temperature=0)(
        Query=example['Query'],
        Type=prediction.Type
    )
    
    verdict = assessment.verdict.strip().upper()
    
    print(f"Query: {example['Query']}\nPredicted: {prediction.Type}\nVerdict: {verdict}\n")
    
    return int(verdict == "YES")


In [31]:
# Initialize the model module
generator = QueryTypePredictor()

print(generator(Query="Hello"))

Prediction(
    reasoning='The query "Hello" is not a specific question or request, so it cannot be classified into a specific type.',
    Type='None'
)


In [32]:
# Initialize your evaluator for the Use–Tool dataset
evaluator = Evaluate(
    devset=test_examples,
    metric=metric,            # Your updated metric function for Use->Tool evaluation
    display_progress=True,
    display_table=10,         # Adjust for your console width
    num_threads=1,
    provide_traceback=True
)

# Evaluate the ToolPredictor module
results = evaluator(generator)

print(f"\nEvaluation Results: {results}")

Query: Show flights from Islamabad to Bahrain next week
Predicted: Travel Information
Verdict: YES

Average Metric: 1.00 / 1 (100.0%):   0%|          | 0/30 [00:00<?, ?it/s]Query: Does Bookme have physical offices?
Predicted: Factual
Verdict: YES

Query: Show me buses from Faisalabad to Lahore tonight
Predicted: Informational
Verdict: YES

Query: Check buses from Lahore to Gujranwala today
Predicted: Information Request
Verdict: NO

Average Metric: 2.00 / 2 (100.0%):   3%|▎         | 1/30 [00:00<00:01, 17.70it/s]Query: Does Bookme accept credit cards?
Predicted: Informational
Verdict: YES

Query: Search flights Karachi to Singapore this week
Predicted: Travel
Verdict: NO

Average Metric: 4.00 / 6 (66.7%):  17%|█▋        | 5/30 [00:00<00:00, 53.19it/s] Query: Check flights from Islamabad to Riyadh and let me know if cancellation is free.
Predicted: Travel Information
Verdict: YES

Average Metric: 5.00 / 7 (71.4%):  23%|██▎       | 7/30 [00:01<00:05,  4.04it/s]Query: Can I cancel my bus 

2025/07/23 16:42:03 INFO dspy.evaluate.evaluate: Average Metric: 24 / 30 (80.0%)


,Query,example_Type,reasoning,pred_Type,metric
0,Show flights from Islamabad to Bahrain next week,api,The query is asking for flights from Islamabad to Bahrain for the ...,Travel Information,✔️ [1]
1,Does Bookme have physical offices?,static,Bookme is a digital platform that allows users to book appointment...,Factual,✔️ [1]
2,Show me buses from Faisalabad to Lahore tonight,api,The query is asking for information about buses from Faisalabad to...,Informational,✔️ [1]
3,Check buses from Lahore to Gujranwala today,api,The query is asking for information about buses from Lahore to Guj...,Information Request,
4,Does Bookme accept credit cards?,static,"The query is asking about the payment method accepted by Bookme, s...",Informational,✔️ [1]
5,Search flights Karachi to Singapore this week,api,The query is searching for flights from Karachi to Singapore for t...,Travel,
6,Check flights from Islamabad to Riyadh and let me know if cancella...,api,The query is asking to check flights from Islamabad to Riyadh and ...,Travel Information,✔️ [1]
7,Can I cancel my bus ticket online and get refund immediately?,static,The query is asking about the possibility of canceling a bus ticke...,Informational,
8,What is the maximum luggage allowed on buses?,static,"The query is asking about the maximum luggage allowed on buses, wh...",Informational,✔️ [1]
9,Search flights from Karachi to Bangkok this week,api,The query is searching for flights from Karachi to Bangkok for thi...,Travel,



Evaluation Results: 80.0


In [33]:
from dspy.teleprompt import BootstrapFewShot, MIPROv2

from dspy.teleprompt import BootstrapFewShotWithRandomSearch
teleprompter = BootstrapFewShotWithRandomSearch( metric=metric,
    max_labeled_demos=100,  # Reduced from 16 to 8 for speed
    max_bootstrapped_demos=4,  # Reduced from 6
    num_candidate_programs=4,  # Reduced from 6
    num_threads=2)  # Parallel processing)
compiled_pred = teleprompter.compile(generator, trainset=train_examples)

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 4 candidate sets.
Query: Search flights from Lahore to Istanbul this month
Predicted: Travel/Flight Booking
Verdict: YES

Query: Can I use Bookme without an account?
Predicted: Informational
Verdict: YES

Query: Find buses from Islamabad to Peshawar tonight
Predicted: Travel Information
Verdict: YES

Query: Check flights from Karachi to Muscat tomorrow
Predicted: Travel Information
Verdict: YES

Query: How can I know if my booking is confirmed?
Predicted: Informational
Verdict: YES

Query: Check buses from Rawalpindi to Murree today
Predicted: Travel Information
Verdict: YES

Average Metric: 6.00 / 6 (100.0%):   4%|▍         | 5/119 [00:00<00:00, 125.00it/s]Query: Book a hotel in Murree for upcoming Friday with free cancellation.
Predicted: Booking Query
Verdict: YES

Query: Search hotels in Skardu for next weekend
Predicted: Travel Booking
Verdict: YES

Query: Find flights from Islamabad to Dubai this Frid

2025/07/23 16:42:39 INFO dspy.evaluate.evaluate: Average Metric: 103 / 119 (86.6%)



New best score: 86.55 for seed -3
Scores so far: [86.55]
Best score so far: 86.55
  0%|          | 0/119 [00:00<?, ?it/s]Query: Can I use Bookme without an account?
Predicted: static
Verdict: YES

Average Metric: 1.00 / 1 (100.0%):   1%|          | 1/119 [00:01<02:42,  1.38s/it]Query: Search flights from Lahore to Istanbul this month
Predicted: api
Verdict: YES

Average Metric: 2.00 / 2 (100.0%):   2%|▏         | 2/119 [00:02<01:58,  1.01s/it]Query: Show flights from Karachi to Doha next week with refund option.
Predicted: api
Verdict: YES

Average Metric: 3.00 / 3 (100.0%):   3%|▎         | 3/119 [00:03<02:04,  1.08s/it]Query: Find buses from Islamabad to Peshawar tonight
Predicted: api
Verdict: YES

Average Metric: 4.00 / 4 (100.0%):   3%|▎         | 4/119 [00:03<01:30,  1.27it/s]Query: Check flights from Karachi to Muscat tomorrow
Predicted: api
Verdict: YES

Average Metric: 5.00 / 5 (100.0%):   4%|▍         | 5/119 [00:04<01:47,  1.06it/s]Query: How can I know if my booking is con

2025/07/23 16:43:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/07/23 16:43:08 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Query: Is there any referral program?
Predicted: static
Verdict: YES

Average Metric: 21.00 / 21 (100.0%):  18%|█▊        | 21/119 [00:29<06:44,  4.13s/it]Query: Check flights from Lahore to Kuala Lumpur and baggage policy.
Predicted: api
Verdict: YES

Average Metric: 22.00 / 22 (100.0%):  18%|█▊        | 22/119 [00:31<05:35,  3.45s/it]Query: Does Bookme offer discounts on flights?
Predicted: static
Verdict: YES

Average Metric: 23.00 / 23 (100.0%):  18%|█▊        | 22/119 [00:31<05:35,  3.45s/it]

...

Query: Can I cancel train tickets booked via Bookme.pk?
Predicted: static
Verdict: YES

Average Metric: 117.00 / 117 (100.0%):  98%|█████████▊| 117/119 [07:52<00:10,  5.29s/it]Query: Find me cheap flights to Dubai from Karachi
Predicted: api
Verdict: YES

Average Metric: 118.00 / 118 (100.0%):  99%|█████████▉| 118/119 [07:53<00:04,  4.08s/it]

2025/07/23 17:34:13 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Query: Check flights from Lahore to Doha next week
Predicted: api
Verdict: YES

Average Metric: 119.00 / 119 (100.0%): 100%|██████████| 119/119 [07:55<00:00,  3.99s/it]

2025/07/23 17:34:15 INFO dspy.evaluate.evaluate: Average Metric: 119 / 119 (100.0%)



Scores so far: [86.55, 100.0, 100.0, 100.0, 99.16, 100.0, 100.0]
Best score so far: 100.0
7 candidate programs found.


In [ ]:
current_directory = "Agent_Decision"

# Save the generator model with the valid path
compiled_pred.save(current_directory,save_program=True)

In [3]:
current_directory = "Agent_Decision"
loaded_dspy_program = dspy.load(current_directory)

In [25]:
test_example = dspy.Example(input="please tell me the possibility , i will book a ticket and refund after landing")
# I booked a flight yesterday from Lahore to Islamabad, and now I want to cancel it and get a refund
result = loaded_dspy_program(test_example)

print("Answer:", result.Type)

Answer: static
